### *Podemos rodar às segundas-feiras já que as partidas da liga são jogadas aos domingos*

In [1]:
import pandas as pd
from functions import clube_id_e_sessao, pega_dados, conecta_ao_db

In [2]:
sessao, club_id= clube_id_e_sessao()

Clube ID: 79085


### Informações gerais dos jogos

In [3]:
# Conecta ao banco de dados
db= conecta_ao_db('dados_sokker.db')
curs= db.cursor()

#Pega informações únicas das partidas já adicionadas para que não sejam inseridos valores duplicados na base
matches = pd.read_sql_query("SELECT * FROM matches", db)

ids= matches['matchID'].unique()
data_esperada= matches['dateExpected'].unique()
data_iniciada= matches['dateStarted'].unique()

In [4]:
# Id do clube a serem pegas as informações
id_clube= club_id

# Pegando informações das partidas retirando aquelas que já foram pegas
url= f'https://sokker.org/xml/matches-team-{club_id}.xml'
df_matches= pega_dados(url, sessao, search= 'match')

#Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)
df_matches.dropna(inplace= True)

query= 'matchID not in @ids and dateExpected not in @data_esperada and dateStarted not in @data_iniciada'

# Retirando colunas que não salvaremos as informações
df_matches.drop(columns= ['homeTeamName', 'awayTeamName'], 
                inplace= True)

#df_matches= pd.concat([matches, df_matches])

df_matches= df_matches.query(query)
df_matches

,matchID,homeTeamID,awayTeamID,leagueID,round,season,week,day,dateExpected,dateStarted,homeTeamScore,awayTeamScore,supporters,weather,isFinished
33,42315630,36457,79085,51530,0,0,1125,5,2025-01-30 22:00,2025-01-30 22:00,1,1,3580,1,1
34,42315491,42244,79085,3,0,0,1125,6,2025-01-31 19:24,2025-01-31 19:25,2,0,5577,5,1
35,42319759,78652,79085,51530,0,0,1126,0,2025-02-01 22:00,2025-02-01 22:00,0,0,6445,7,1
36,42084957,79085,122169,1349,13,69,1126,1,2025-02-02 21:59,2025-02-02 21:59,2,4,15188,5,1
37,42319703,90579,79085,51530,0,0,1126,2,2025-02-03 22:00,2025-02-03 22:00,0,0,3665,5,1
38,42319886,90579,79085,51530,0,0,1126,3,2025-02-04 22:00,2025-02-04 22:00,0,1,5151,5,1
39,42084941,79085,23755,1349,14,69,1126,4,2025-02-05 21:58,2025-02-05 21:59,2,5,17618,4,1
40,42320703,79085,90579,51530,0,0,1126,5,2025-02-06 22:00,2025-02-06 22:00,1,2,2713,4,1


In [5]:
if len(df_matches) > 0:
    
    # Adiciona dados à tabela
    df_matches.to_sql('matches', con=db, if_exists='append', index= False)

In [6]:
#df_matches = pd.read_sql_query("SELECT * FROM matches", db)
len(df_matches)

8

### Estatísticas dos jogos

In [7]:
#Pega informações únicas das estatísticas já adicionadas para que não sejam inseridos valores duplicados na base
stats = pd.read_sql_query("SELECT * FROM matchStats", db)

ids= stats['matchID'].unique()
team= stats['teamID'].unique()
terco_defensivo= stats['timeOnHalf'].unique()
posse_de_bola= stats['timePossession'].unique()

In [8]:
#lista que guardará as estatísticas de cada partida
list_stats= list()

# Pegando estatísticas

for ID in df_matches['matchID'].unique():
    
    url= f'https://sokker.org/xml/match-{ID}.xml'
    df= pega_dados(url, sessao, search= 'teamStats')
    df['matchID']= ID
    list_stats.append(df)
    


df_stats= pd.concat([stats for stats in list_stats])

# Retirando valores nulos (partidas ainda não jogadas) e duplicados (partidas já adicionadas na base)

df_stats.dropna(inplace= True)

query= 'matchID not in @ids and teamID not in @team and timeOnHalf not in @terco_defensivo and timePossession not in @posse_de_bola'

df_stats= df_stats.query(query)

df_stats

,teamID,timeOnHalf,timePossession,offsides,shoots,fouls,yellowCards,redCards,tacticName,ratingScoring,ratingPassing,ratingDefending,matchID
0,36457,31559,32994,0,4,0,0,0,WS POG,7,8,5,42315630
1,79085,38393,36958,1,7,1,0,1,5-3-2c,9,9,9,42315630
0,42244,27658,37683,0,13,1,0,0,rombo meva,7,5,6,42315491
1,79085,42389,32364,4,2,4,1,0,4-3-3aRU,5,7,7,42315491
0,78652,43717,31228,0,1,2,1,0,4-1-2-1-2 SLDSLA,6,6,5,42319759
1,79085,26675,39164,1,19,1,0,0,3-4-3aPS,7,9,8,42319759
0,79085,48142,33963,2,7,4,1,0,3-4-3aPS,9,9,10,42084957
1,122169,22439,36618,1,11,1,0,0,Nó Tático,10,7,9,42084957
0,90579,32599,32973,0,5,6,1,0,5 - 4,9,10,12,42319703
1,79085,37359,36985,0,2,2,1,0,5-4-1a,8,9,10,42319703


In [9]:
# Adicionando informações se houver alguma nova
if len(df_stats) > 0:
    
    # Adiciona dados à tabela
    df_stats.to_sql('matchStats', con=db, if_exists='append', index= False)

In [10]:
# Conferindo
stats = pd.read_sql_query("SELECT * FROM matchStats", db)
len(stats)

1548

### Estatísticas da Liga

In [11]:
#Pega informações ligas já adicionadas para que não sejam inseridos valores duplicados na base
league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)

ids= league_stats['leagueID'].unique()
team= league_stats['teamID'].unique()
rodada=  league_stats['round'].unique()

In [12]:
#Pegar o id da liga desejada entrando na liga na página do sokker
league_id= 1349

url= f'https://sokker.org/xml/league-{league_id}.xml'
df_league_stats= pega_dados(url, sessao, search= 'teams', tree_top= True)

In [13]:
df_league_stats['leagueID']= league_id
df_league_stats

,teamID,round,points,wins,draws,losses,goalsScored,goalsLost,rankTotal,leagueID
0,47841,14,39,13,0,1,73,12,39561073013010,1349
1,6295,14,34,11,1,2,73,24,34549073011009,1349
2,79866,14,32,10,2,2,43,12,32531043010005,1349
3,23755,14,29,9,2,3,65,20,29545065009011,1349
4,78671,14,23,7,2,5,47,22,23525047007007,1349
5,89569,14,22,7,1,6,31,32,22499031007002,1349
6,79085,14,19,5,4,5,34,22,19512034005006,1349
7,124139,14,18,6,0,8,39,42,18497039006008,1349
8,122169,14,17,5,2,7,57,45,17512057005004,1349
9,48541,14,9,3,0,11,16,87,9429016003001,1349


In [14]:
# Adicionando informações 
if len(df_league_stats) > 0:
    
    # Adiciona dados à tabela
    df_league_stats.to_sql('leagueStats', con=db, if_exists='append', index= False)

league_stats = pd.read_sql_query("SELECT * FROM leagueStats", db)
len(league_stats)

### Liga informação geral

In [15]:
#Pega informações das ligas já adicionadas para que não sejam inseridos valores duplicados na base
leagues = pd.read_sql_query("SELECT * FROM league", db)

ids= leagues['leagueID'].unique()
league_name= leagues['name'].unique()

In [16]:
#Pegando dados da liga
df_league= pega_dados(url, sessao, search= 'info')

#Retirando coluna que não será utilizada
df_league.drop(columns= ['userID', 'round'], inplace= True)

#Retirando valores duplicados (se as estatísticas já tiverem sido adicionadas na base)
query= 'leagueID not in @ids and name not in @league_name'
df_league= df_league.query(query)

df_league

,leagueID,name,countryID,division,season,type,isOfficial,isCup


In [17]:
# Adicionando informações se for uma liga ainda não adicionada
if len(df_league) > 0:
    
    # Adiciona dados à tabela
    df_league.to_sql('league', con=db, if_exists='append', index= False)

In [18]:
league = pd.read_sql_query("SELECT * FROM league", db)
len(league)

4

In [19]:
# Salvando alterações
db.commit()

# Fechando conexão com a base de dados
db.close()